In [1]:
!pip install emoji
!pip install tensorflow
!pip install transformers

     |████████████████████████████████| 175 kB 5.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=13497d04e12f0b5244effc1cf45f5a75e26fd3261321ebc04b496b33b6e5b7cb
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 462 kB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 83.5 MB/s 
     |████████████████████████████████| 6.6 MB 61.1 MB/s 
     |████████████████████████████████| 596 kB 48.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

model = TFAutoModelForSequenceClassification.from_pretrained("rabindralamsal/finetuned-bertweet-sentiment-analysis")

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/515M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at rabindralamsal/finetuned-bertweet-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [4]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
link = 'https://drive.google.com/file/d/17-X8-Fg4B11bqxt_fZ5B-X2WzeuRky0f/view?usp=sharing' 

In [6]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Train_Feb.csv') 
 
df = pd.read_csv('Train_Feb.csv')
print(df)

                                                      text
0        @k_c12345 Brother this called link 👇\nhttps://...
1        Ukraine invasion: Who is President Volodymyr Z...
2        So it now looks like we're in a new phase of t...
3        @vanderbellen @karlnehammer TOP RUSSIAN AGGRES...
4        @ZelenskyyUa @EmmanuelMacron Ukraine won't fal...
...                                                    ...
1782546  @UNESCO Dear UNESCO. I'm a peaceful citizen of...
1782547  This is not Ukraine vs Russia war. This is a c...
1782548  Russia 🇷🇺 is warn... Don't provoke the 4nation...
1782549  Africa Report: How Africa factors in #RussiaUk...
1782550  @BanteSaket @LukeMackay13 @Reporter_indian I l...

[1782551 rows x 1 columns]


In [7]:
df['text'] = df['text'].astype('str') 

In [8]:
#df_sample = df.sample(n=50000, random_state=2)

In [9]:
#df_sample

In [10]:
docs = list(df.loc[:, "text"].values)

In [11]:
#get index for halving dataset at index: 178255
from math import floor
floor(len(docs) / 10)


178255

In [12]:
178255 * 3

534765

In [13]:
docs = docs[356510:534765]

In [14]:
from tqdm import tqdm
scores = []

for i in tqdm(range(len(docs))):
    input = tokenizer.encode(docs[i], return_tensors="tf")
    output = model.predict(input)[0]
    prediction = tf.nn.softmax(output, axis=1).numpy()
    sentiment = np.argmax(prediction)
    
    scores.append({"Label": sentiment,
                       "Negative": prediction[0,0],
                       "Neutral": prediction[0,1],
                       "Positive": prediction[0,2]
                  })

100%|██████████| 178255/178255 [2:50:20<00:00, 17.44it/s]


In [15]:
df = df[356510:534765]

In [16]:
sentiments_score = pd.DataFrame.from_dict(scores)
df = df.join(sentiments_score.set_index(df.index))

In [17]:
df

,text,Label,Negative,Neutral,Positive
356510,Russia's invasion of Ukraine has sparked a hum...,0,0.815030,0.180699,0.004271
356511,@Atalanta_BC Can you tell this son of a bitch ...,0,0.975286,0.018263,0.006452
356512,@DeutscheWelle USA and Germany made sure commo...,0,0.858261,0.138333,0.003407
356513,-It appears Russia has issued a NOTAM for the ...,1,0.040979,0.948854,0.010167
356514,'The stakes could not possibly be higher' | Wi...,0,0.508059,0.484711,0.007230
...,...,...,...,...,...
534760,@dhume @RShivshankar Got a fwded msg from a RW...,1,0.206330,0.777557,0.016114
534761,"""#Russia has never been under threat from @NAT...",1,0.228273,0.751658,0.020069
534762,@akarl_smith Tucker Carlson never supported Pu...,0,0.967141,0.029824,0.003035
534763,SWIFT is absolutely necessary #Germans #OlafSc...,0,0.640962,0.298937,0.060101


In [18]:
from google.colab import drive
drive.mount('drive')

df.to_csv('finetuned_Bertweet_SA_Feb_Train_p3.csv')
!cp finetuned_Bertweet_SA_Feb_Train_p3.csv 'drive/My Drive/Colab Notebooks'

Mounted at drive
